In [63]:
# !python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 46.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [64]:
import os
import re
import spacy
import awswrangler as wr
import pandas as pd

## Carga de datos

In [32]:
df_hogar_mania = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/hogar_mania')
df_antena3 = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/antena3')
df_saber_vivir = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/saber_vivir')
df_el_mundo = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/el_mundo')

### Hogar Mania

Obtener categorias

In [33]:
categorias_hogar_mania = df_hogar_mania['categoria'].unique()
categorias_hogar_mania

<StringArray>
[             'postres',           'aperitivos',   'ensaladas-verduras',
     'arroces-cereales',        'pastas-pizzas',         'sopas-cremas',
            'legumbres',               'carnes',    'pescados-mariscos',
         'setas-hongos',               'huevos',               'salsas',
            'desayunos',          'recetas-pan',              'bebidas',
             'cocteles', 'cocina-internacional']
Length: 17, dtype: string

Eliminar nulos

In [34]:
df_hogar_mania.shape

(7464, 5)

In [35]:
df_hogar_mania.dropna(subset=["ingredientes"], inplace=True)

In [36]:
df_hogar_mania.shape

(7228, 5)

In [37]:
df_hogar_mania['total_ingredientes'] = df_hogar_mania['ingredientes'].apply(len)
df_hogar_mania[df_hogar_mania['total_ingredientes'] == 0].shape

(18, 6)

In [38]:
df_hogar_mania = df_hogar_mania[df_hogar_mania['total_ingredientes'] != 0]

In [39]:
df_hogar_mania[df_hogar_mania.duplicated(subset=['link'], keep=False)]['categoria'].unique()

<StringArray>
[             'postres',           'aperitivos',   'ensaladas-verduras',
     'arroces-cereales',        'pastas-pizzas',         'sopas-cremas',
            'legumbres',               'carnes',    'pescados-mariscos',
               'huevos',            'desayunos',          'recetas-pan',
              'bebidas',             'cocteles', 'cocina-internacional']
Length: 15, dtype: string

In [40]:
df_hogar_mania_double = df_hogar_mania[df_hogar_mania.duplicated(subset=['link'], keep=False)]
df_hogar_mania_double.groupby(by='link')['categoria'].apply(list)

link
https://www.hogarmania.com//cocina/recetas/aperitivos/desayuno-real-cereal-37095.html                                        [aperitivos, desayunos]
https://www.hogarmania.com//cocina/recetas/aperitivos/jocoque-10518.html                                          [aperitivos, cocina-internacional]
https://www.hogarmania.com//cocina/recetas/aperitivos/molletes-mexicana-36967.html                                           [aperitivos, desayunos]
https://www.hogarmania.com//cocina/recetas/aperitivos/zumo-papaya-naranja-pomelo-25672.html                                    [aperitivos, bebidas]
https://www.hogarmania.com//cocina/recetas/arroces-cereales/arroz-colores-salsa-agridulce-asia-2684.html    [arroces-cereales, cocina-internacional]
                                                                                                                              ...                   
https://www.hogarmania.com//cocina/recetas/recetas-pan/bagels-rosquillas-18773.html                  

In [41]:
df_hogar_mania.drop_duplicates(subset=['link'], keep='first', inplace=True)

In [42]:
df_hogar_mania.shape

(7056, 6)

In [ ]:
def extract_words(text):
    words = re.findall(r'\b[a-zA-Z]+\b', text)
    return ' '.join(words)

In [60]:
df_hogar_mania['titulo_link'] = (
    df_hogar_mania['link']
    .apply(os.path.basename)
    .apply(os.path.splitext)
    .apply(lambda text: text[0])
    .apply(extract_words)
)
df_hogar_mania[['titulo', 'titulo_link']]

,titulo,titulo_link
0,"Buñuelos de viento fáciles, la receta tradicio...",bunuelos
1,Corona de hojaldre de Navidad,corona navidad
2,Cafés de Navidad: Gingerbread Latte y Pumpkin ...,cafes navidad gingerbread pumpkin spice latte
4,Mazapanes de Navidad,mazapanes navidad
6,"Churros en freidora de aire, ¡más fácil imposi...",churros freidora aire
...,...,...
7384,"Tarta fría de limón de Eva Arguiñano, la más c...",tarta fria limon
7385,Receta de mochis japoneses caseros,receta mochis japoneses caseros
7386,"Receta de helado de pistacho, paso a paso",helado pistacho casero
7387,Cómo hacer gachas dulces con los pasos de Karl...,gachas dulces


In [65]:
nlp = spacy.load('es_core_news_sm')

In [176]:
def extract_ingredients(text):
    doc = nlp(text.lower())
    unnecessary_bow = [
        "gr",
        "g",
        "kg",
        "ml",
        "l",
        "oz",
        "lb",
        "gr.",
        "g.",
        "kg.",
        "ml.",
        "l.",
        "oz.",
        "lb.",
        "kilo",
        "litro",
        "gramo",
        "vaso",
        "taza",
        "tazón",
        "pizca",
        "cucharada",
        "cucharadas",
        "cucharadita",
        "ralladura",
    ]
    admitted_class_words = ["NOUN", "PROPN", "PRON", "ADJ"]
    filtered_tokens = [
        token.text
        for token in doc
        if not token.like_num
        and not token.is_stop
        and not token.is_punct
        and token.pos_ in admitted_class_words
        and token.text not in unnecessary_bow
    ]
    return " ".join(filtered_tokens)

In [177]:
df_hogar_mania['ingredientes_limpios'] = df_hogar_mania['ingredientes'].apply(lambda ingrs: [extract_ingredients(ingr) for ingr in ingrs])

0       [harina, mantequilla, agua, azúcar, huevos, li...
1       [láminas hojaldre, chocolate negro, mantequill...
2       [azúcar moreno suave, jengibre molido, nuez mo...
4       [almendra molida, leche condensada, limón, , a...
6       [churros congelados, aceite oliva girasol, azú...
                              ...                        
7384                      [galletas, mantequilla fundida]
7385    [harina arroz glutinoso, agua, azúcar recetas ...
7386    [g. pasta pistachos comercial casera, g. pista...
7387    [leche, harina, azúcar, semillas anís, , limón...
7388    [variedad chuches gusto botes kilo euros, palo...
Name: ingredientes, Length: 7056, dtype: object